# 35 - Transformaciones de Datos de Variables Categóricas


* En este Notebook vamos a ver como ***transformar los datos de variable categóricas para que los algorítmos de aprendizaje*** (que solo entienden de "números" y no de Strings) ***puedan usar esos datos*** para generar los modelos.


* Vamos a ver las siguientes Transformaciones:
<span></span><br>
    1. [Codificación Binaria](#M1)
<span></span><br>
    2. [Codificación One Hot Encode](#M2)
<span></span><br>
    3. [Label Encoder](#M3)


* Los típos de variables estadísticas que nos podemos encontrar son las siguientes:


<img src="./imgs/35_01_var.png" style="width: 800px;"/>



* ***Los algorítmos de aprendizaje no tienen ningún problema a la hora de trabajar con variables cuantitativas ya que estas siendo discretas (por ejemplo: 1, 2, 3, ... N) o continuas ($-\infty$ a $\infty$)*** pueden trabajar con ellas.


* Sin embargo las ***variables Cualitativas o Categorias (por ejemplo {Madrid, Barcelona, Valencia}, {Primera, Segunda, Tercera}) son Strings y los algoritmos de aprendizaje no pueden trabajar con ellas por tanto hay que transformarlas a números***.


* Distinguimos dos tipos de variables Cualitativas o Categóricas que son:
<span></span><br><br>
    + ***Nominales***: No admiten orden, por ejemplo {Hombre, Mujer} o {Madrid, Barcelona, Valencia}
<span></span><br><br>
    + ***Ordinales***: Admiten un orden entre ellas como por ejemplo {Primera, Segunda, Tercera}


* Dependiendo del tipo de variables Cualitativas o Categóricas debemos de realizar una transformación un otra.


* ***Variables Nominales***: Debemos de codificar la salida de tal manera que podamos distiguir los tipos. Esto lo podemos hacer con una codificación binária en caso de tener 2 valores o con un "One Hot Encode" en caso de tener más valores. Veamos unos ejemplos:
 
    * ***Codificación Binaria***:
    
|Genero|Codificación|
|---|---|
|Masculino|0|
|Femenino|1|

   * ***Codificación One Hot Encode***:
   
|Ciudad|Madrid|Barcelona|Valencia|
|---|---|---|---|
|Madrid|1|0|0|
|Barcelona|0|1|0|
|Valencia|0|0|1|


* ***Variables Ordinales***: Al ser unas variables que admiten orden, podemos transformar los String a variables numéricas, poniendo el orden que queramos definir. Esta transformación se conoce como "Label Encode". Veamos un ejemplo:

    * ***Codificación Label Encode***:
    
|Categoria|Cat_Ordenada_1|Cat_Ordenada_2|
|---|---|---|
|Primera|1|3|
|Segunda|2|2|
|Tercera|3|1|


* Veamos a continuación como hacer este tipo de Transformaciones:



<hr>

## Carga de Datos

* En primer lugar vamos a definir un DataFrame de ejemplo con el que poder ver estas transformaciones.


* El siguiente DataFrame tiene las siguientes variables que las clasificaremos como:

    + ***Edad***: Variable Continua
    + ***Genero***: Variable Categorica Nominal (2 valores)
    + ***Ciudad_embarque***:Variable Categorica Nominal (>2 valores)
    + ***Clase***: Variable Categorica Ordinal



In [1]:
import pandas as pd

df = pd.DataFrame(data={
    'edad': [20, 30, 40, 50, 60],
    'genero': ['Mas', 'Mas', 'Fem', 'Fem', 'Fem'],
    'ciudad_embarque': ['Cherboarg', 'Queenstown', 'Southampton', 'Cherboarg', 'Queenstown'],
    'clase': ['Cuarta', 'Tercera', 'Primera', 'Segunda', 'Segunda']
})

df

,edad,genero,ciudad_embarque,clase
0,20,Mas,Cherboarg,Cuarta
1,30,Mas,Queenstown,Tercera
2,40,Fem,Southampton,Primera
3,50,Fem,Cherboarg,Segunda
4,60,Fem,Queenstown,Segunda


<hr>


# <a name="M1">1. Codificación Binaria</a>


* Vamos a transformar la variable genero a una variable binaria.


* Esto lo podemos hacer con la clase **"LabelBinarizer()"** de scikit-learn de la siguiente manera:

In [2]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()
df['genero_binario'] = lb.fit_transform(df['genero'])
df[['genero', 'genero_binario']]

,genero,genero_binario
0,Mas,1
1,Mas,1
2,Fem,0
3,Fem,0
4,Fem,0


<hr>


# <a name="M2">2. Codificación One Hot Encode</a>


* Vamos a transformar la variable "ciudad_embarque" en tantas nuevas variables como elementos tenga.


* Esto lo podemos hacer con la clase **"OneHotEncoder()"** de scikit-learn de la siguiente manera:

In [3]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
x = df[['ciudad_embarque']].values
x_one_hot = ohe.fit_transform(x).toarray()
x_one_hot

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.]])

* Como vemos nos devuelve una matriz con las codificaciones de cada una de las filas del DataFrame (transformadas a un numpy array).


* Vamos ahora a pasar estas nuevas variables al DataFrame:


In [4]:
df_one_hot = pd.DataFrame(x_one_hot, columns = ["ciudad_embarque_"+str(int(i)) for i in range(x_one_hot.shape[1])])
df_all = pd.concat([df, df_one_hot], axis=1)
df_all

,edad,genero,ciudad_embarque,clase,genero_binario,ciudad_embarque_0,ciudad_embarque_1,ciudad_embarque_2
0,20,Mas,Cherboarg,Cuarta,1,1.0,0.0,0.0
1,30,Mas,Queenstown,Tercera,1,0.0,1.0,0.0
2,40,Fem,Southampton,Primera,0,0.0,0.0,1.0
3,50,Fem,Cherboarg,Segunda,0,1.0,0.0,0.0
4,60,Fem,Queenstown,Segunda,0,0.0,1.0,0.0


### Codificación One Hot Encode en Pandas


* Esta transformación también nos la permite hacer la librería de Pandas de la siguiente manera:

In [5]:
df_dummy = pd.get_dummies(df['ciudad_embarque'])
df_dummy

,Cherboarg,Queenstown,Southampton
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0
4,0,1,0


* Concatenamos la codificación resultante al DataFrame

In [6]:
df = pd.concat([df, df_dummy], axis=1)
df

,edad,genero,ciudad_embarque,clase,genero_binario,Cherboarg,Queenstown,Southampton
0,20,Mas,Cherboarg,Cuarta,1,1,0,0
1,30,Mas,Queenstown,Tercera,1,0,1,0
2,40,Fem,Southampton,Primera,0,0,0,1
3,50,Fem,Cherboarg,Segunda,0,1,0,0
4,60,Fem,Queenstown,Segunda,0,0,1,0


<hr>



# <a name="M3">3. Label Encoder</a>


* Vamos a transformar la variable "clase" a una variable numerica.


* Esto lo podemos hacer con la clase **"LabelEncoder()"** de scikit-learn de la siguiente manera:

In [7]:
from sklearn.preprocessing import LabelEncoder

df['clase_endoce'] = LabelEncoder().fit_transform(df['clase'])
df[['clase', 'clase_endoce']]

,clase,clase_endoce
0,Cuarta,0
1,Tercera,3
2,Primera,1
3,Segunda,2
4,Segunda,2


* Con la clase **"LabelEncoder()"** de Scikit tenemos el problema de que nos pone a cada elemento un valor en función del orden alfabetico.


* En el caso de trabajar con variables categóricas ordinales, nos interesa darle un sentido ordinal a los valores de la variable, por lo que vamos a asignarle los valores de la siguiente manera:

In [8]:
df['clase_endoce'] = df['clase'].apply(lambda x: 1 if x == 'Primera' 
                                       else (2 if x == 'Segunda' 
                                             else (3 if x == 'Tercera' else 4)))
df[['clase', 'clase_endoce']]

,clase,clase_endoce
0,Cuarta,4
1,Tercera,3
2,Primera,1
3,Segunda,2
4,Segunda,2


<hr>


# Resultado Final:


* Tras realizar todas estas transformaciones ya tendríamos el Dataset transformado correctamente para que pueda ser utilizado por un Algoritmo de Aprendizaje:


In [9]:
df[['edad', 'genero_binario', 'Cherboarg', 'Queenstown', 'Southampton', 'clase_endoce']]

,edad,genero_binario,Cherboarg,Queenstown,Southampton,clase_endoce
0,20,1,1,0,0,4
1,30,1,0,1,0,3
2,40,0,0,0,1,1
3,50,0,1,0,0,2
4,60,0,0,1,0,2


<hr>

*Este Notebook ha sido desarrollado por **Ricardo Moya García** y registrado en Safe Creative como ***Atribución-NoComercial-CompartirIgual***.*

<img src="./imgs/CC_BY-NC-SA.png" alt="CC BY-NC">